<a href="https://colab.research.google.com/github/daigokk/wave/blob/main/examples/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プログラミング体験：音
プログラムで音を作ります。<br>
- 左右のスピーカーから異なる音を出すことで、ステレオ感を演出してみましょう。スピーカーがモノラル(1ch)の場合、ステレオ(2ch)は強制的にモノラル再生となります。イヤホンやヘッドホンを使うと、ステレオ感を感じやすくなります。<br>
- 波形を変えることで音色が変わります。波形と音色の関係を体感してみましょう。<br>



## 準備1：セルマジックの定義
ColabでC言語を簡単に使う設定を行います。下のセルを実行してください。<br>
「Shift+Enter」またはセル左側の右を向いた三角形のアンコンを押すと実行できます。

In [ ]:
# Python, セルマジックcppの定義
from IPython.display import Audio
from IPython.core.magic import register_cell_magic
@register_cell_magic
def cpp(line, cell):
    program = 'a.out'
    option = ''
    if line == '':
        line = '_temp.cpp'
    with open(line, 'w') as f:
        f.write(cell)
    ipy = get_ipython()
    ret = ipy.getoutput(f'g++ {line} -o {program} {option}')
    if len(ret) != 0:
        for error in ret:
            print(error)
    else:
        ret = ipy.getoutput(f'./{program}')
        for error in ret:
            print(error)
        if len(ret) == 0:
            return Audio("test.wav")
    return

## 準備2：waveヘッダファイルをダウンロード
音を保存するプログラムをダウンロードします。

In [ ]:
# Waveファイルを作るためのヘッダファイルをダウンロードします。
!wget https://raw.githubusercontent.com/daigokk/wave/main/wave.hpp

## 1 waveファイルの作製
「ドレミーーーーーーー」の音を作ります。

In [ ]:
%%cpp
#include "wave.hpp"
#include <math.h>
#define PI acos(-1)

int main(void)
{
    wave pcm(10);
    for(int i = 0; i < pcm.size; i++)
    {
        double amp_right = 1;
        double amp_left = 1;
        double freq;
        if(i < pcm.fs * 1) freq = scale::C4;
        else if(i < pcm.fs * 2) freq = scale::D4;
        else if(i < pcm.fs * 3) freq = scale::E4;
        pcm.s[0][i] = amp_right * sin(2 * PI * freq * i / pcm.fs);
        pcm.s[1][i] = amp_left *  sin(2 * PI * freq * i / pcm.fs);
    }
    pcm.write("test.wav");
    return 0;
}

## 2 左右のチャンネルの音の大きさを変える
左右のチャンネルに位相の異なるAmplitude Modulation (AM変調)をかけて、ステレオ感を演出してみます。

In [ ]:
%%cpp
#include "wave.hpp"
#include <math.h>
#define PI acos(-1)

int main(void)
{
    wave pcm(10);
    for(int i = 0; i < pcm.size; i++)
    {
        double amp_right = sin(2 * PI * 0.25 * i / pcm.fs + PI / 2);
        double amp_left = sin(2 * PI * 0.25 * i / pcm.fs);
        double freq;
        if(i < pcm.fs * 1) freq = scale::C4;
        else if(i < pcm.fs * 2) freq = scale::D4;
        else if(i < pcm.fs * 3) freq = scale::E4;
        pcm.s[0][i] = amp_right * sin(2 * PI * freq * i / pcm.fs);
        pcm.s[1][i] = amp_left *  sin(2 * PI * freq * i / pcm.fs);
    }
    pcm.write("test.wav");
    return 0;
}

## 3 音の移動
左右のチャンネルに位相の異なるガウシアンを掛けて、音源が右から左へ移動するのを模擬してみます。

In [ ]:
%%cpp
#include "wave.hpp"
#include <math.h>
#define PI acos(-1)

int main(void)
{
    wave pcm(10);
    for(int i = 0; i < pcm.size; i++)
    {
        double amp_right = exp(-pow(i - pcm.fs * 5, 2) / (2 * pow(pcm.size/10, 2)));
        double amp_left = exp(-pow(i - pcm.fs * 3, 2) / (2 * pow(pcm.size/10, 2)));
        double freq = scale::E4;
        pcm.s[0][i] = amp_right * sin(2 * PI * freq * i / pcm.fs);
        pcm.s[1][i] = amp_left *  sin(2 * PI * freq * i / pcm.fs);
    }
    pcm.write("test.wav");
    return 0;
}

## 4 Chart
波形をChartにしてみます。

In [ ]:
!wget https://raw.githubusercontent.com/lava/matplotlib-cpp/master/matplotlibcpp.h
# Python, セルマジックcppの定義
from IPython.display import display, Image
from IPython.core.magic import register_cell_magic
@register_cell_magic
def cpp(line, cell):
    program = 'a.out'
    option = '-I/usr/include/python3.7 -lpython3.7m -I/usr/local/lib/python3.7/dist-packages/numpy/core/include'
    if line == '':
        line = '_temp.cpp'
    with open(line, 'w') as f:
        f.write(cell)
    ipy = get_ipython()
    ret = ipy.getoutput(f'g++ {line} -o {program} {option}')
    if len(ret) != 0:
        for error in ret:
            print(error)
    else:
        ret = ipy.getoutput(f'./{program}')
        for error in ret:
            print(error)
        if len(ret) != 0 and ret[0] == '<Figure size 640x480 with 1 Axes>':
            display(Image('chart.png'))
            return Audio("test.wav")
    return

In [ ]:
%%cpp
#include "wave.hpp"
#include <math.h>
#define PI acos(-1)
#include <vector>
#include "matplotlibcpp.h"
namespace plt = matplotlibcpp;

void chart(wave& pcm);

int main(void)
{
    wave pcm(10);
    for(int i = 0; i < pcm.size; i++)
    {
        double amp_right = exp(-pow(i - pcm.fs * 5, 2) / (2 * pow(pcm.size/10, 2)));
        double amp_left = exp(-pow(i - pcm.fs * 3, 2) / (2 * pow(pcm.size/10, 2)));
        double freq = scale::E4;
        pcm.s[0][i] = amp_right * sin(2 * PI * freq * i / pcm.fs);
        pcm.s[1][i] = amp_left *  sin(2 * PI * freq * i / pcm.fs);
    }
    pcm.write("test.wav");
    chart(pcm);
    return 0;
}

void chart(wave& pcm)
{
    std::vector<double> t(pcm.size), left(pcm.size), right(pcm.size);
    for(int i = 0; i < pcm.size; i++)
    {
        t[i] = (double)i / pcm.fs;
        right[i] = pcm.s[0][i];
        left[i] = pcm.s[1][i];
    }
    plt::named_plot("Right", t, right);
    plt::named_plot("Left", t, left);
    plt::legend();
    plt::xlabel("Time (s)");
    plt::save("chart.png");
    plt::show();
}

## 5 波形と音色
波形を変えた時の音色を確認しましょう。次のセルの14行目を以下のようにすると波形が変わります。<br>
正弦波：`double k = 1;`<br>
のこぎり波：`double k = n`<br>
$\sum_{n=1}^{100}{\frac{\sin(2\pi{fnt})}{n}}$<br>
矩形波：`double k = 2 * n - 1;`<br>
$\sum_{n=1}^{100}{\frac{\sin(2\pi{f(2n-1)}t)}{2n-1}}$<br>

In [ ]:
%%cpp
#include "wave.hpp"
#include <math.h>
#include <vector>
#define PI acos(-1)
#include "matplotlibcpp.h"
namespace plt = matplotlibcpp;
void chart(wave& pcm);
int main(void)
{
    wave pcm(5);
    for(int n = 1; n <= 100; n++)
    {
        double k = 1;
        for(int i = 0; i < pcm.size; i++)
        {
            double amp_right = 1.0 / k;
            double amp_left = 1.0 / k;
            double s;
            if(i < pcm.fs * 0.5) s = sin(2 * PI * scale::C4 * k / pcm.fs * i) / k;
            else if(i < pcm.fs * 1) s = sin(2 * PI * scale::E4 * k / pcm.fs * i) / k;
            else if(i < pcm.fs * 1.5) s = sin(2 * PI * scale::G4 * k / pcm.fs * i) / k;
            else s = (sin(2 * PI * scale::C4 * k / pcm.fs * i)
                + sin(2 * PI * scale::E4 * k / pcm.fs * i)
                + sin(2 * PI * scale::G4 * k / pcm.fs * i)) / k;
            pcm.s[0][i] += s;
            pcm.s[1][i] += s;
        }
    }
    
    double max = 0;
    for(int i = 0; i < pcm.size; i++)
    {
        for(int ch = 0; ch < pcm.chs; ch++)
        {
            if(max < abs(pcm.s[ch][i])) max = pcm.s[ch][i];
        }
    }
    for(int i = 0; i < pcm.size; i++)
    {
        for(int ch = 0; ch < pcm.chs; ch++)
        {
            pcm.s[ch][i] /= max;
        }
    }
    pcm.write("test.wav");
    chart(pcm);
    return 0;
}

void chart(wave& pcm)
{
    std::vector<double> t(pcm.size), left(pcm.size), right(pcm.size);
    for(int i = 0; i < pcm.size; i++)
    {
        t[i] = (double)i / pcm.fs;
        right[i] = pcm.s[0][i];
        left[i] = pcm.s[1][i];
    }
    plt::xlim(0.0,0.01);
    plt::named_plot("Right", t, right);
    plt::named_plot("Left", t, left);
    plt::legend();
    plt::xlabel("Time (s)");
    plt::save("chart.png");
    plt::show();
}